In [30]:
from urllib import request
from bs4 import BeautifulSoup
import urllib.request
import requests

# URLごとにスクレイピングを行う
def scraping(url):
    #ページに複数ある対象部分を取得する
    html = request.urlopen(url)
    soup = BeautifulSoup(html, fromEncoding="euc-jp")
    drama_info_tables = soup.select('.tableborder.ar')
    # cast
    tmp_drama_info_dict = {}
    for dt in drama_info_tables:
        drama_title = dt.select('.red')
        cast_staff = dt.select('.left.pl5px')
        
        # <フジテレビ>などを消す
        drama_title_key = reshape_title(drama_title[0].text)
        # キャストスタッフは調整中
#         import pdb; pdb.set_trace()
        cast_staff_value = reshape_cast_staff(str(cast_staff[0]))
        
        tmp_drama_info_dict[drama_title_key] = cast_staff_value
#         print (drama_title[0])
#         print ("hoge")
#         print (drama_title[0].text)
    
    return tmp_drama_info_dict

In [31]:
# 以下yymmを設定する配列
yymm_arr = ["1907", "1904"]
# urlを配列で作成
urls_arr =[]
for yymm in yymm_arr:
    preurl = "https://artv.info/ar"
    epiurl = ".html"
    # urlを合体(ex: https://artv.info/ar1707.html )
    url = preurl + yymm +epiurl
    urls_arr.append(url)

In [45]:
def reshape_cast_staff (tmp_cast_staff): 
    tmp_splited = tmp_cast_staff.split("<br/>")
    tmp_splited = [tmp_ele for tmp_ele in tmp_splited if tmp_ele]
    import re
    p = re.compile(r"<[^>]*?>|\n|\r|\t")

    info_arr = []
    for tmp_text in tmp_splited:
        #import pdb; pdb.set_trace()
        tmp_text_after = p.sub("",tmp_text)
        if tmp_text_after:
            info_arr.append(tmp_text_after)
        
    return (info_arr)

In [46]:
def reshape_title (tmp_title): 
    
    import re
    p = re.compile(r"\u3000＜\w+＞")

    return(p.sub("",tmp_title))

In [47]:
###########
# 実行本体
#
drama_info_dict = {}
for tmp_url in urls_arr:
    tmp_drama_info_dict = scraping(tmp_url)
    drama_info_dict.update(tmp_drama_info_dict)

import pprint
pprint.pprint(drama_info_dict)

/Users/yoshimurahiroyuki/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:164: UserWarning: The "fromEncoding" argument to the BeautifulSoup constructor has been renamed to "from_encoding."
  'has been renamed to "%s."' % (old_name, new_name))


{'Heaven？\u3000〜ご苦楽レストラン〜': ['石原さとみ、福士蒼汰、',
                             '志尊淳、田口浩正、内田慈、大重わたる、',
                             '松本ししまる、勝村政信、矢柴俊博、上田遥、',
                             '舘ひろし、段田安則、岸部一徳 etc',
                             '原作 ： 佐々木倫子「Heaven？ ご苦楽レストラン」',
                             '脚本 ： 吉田恵里香',
                             'Ｐ ： 瀬戸口克陽',
                             '演出 ： 木村ひさし etc',
                             '主題歌 ： あいみょん『真夏の夜の匂いがする』'],
 'TWO WEEKS': ['三浦春馬、芳根京子、',
               '比嘉愛未、三浦貴大、稲垣来泉、近藤公園、',
               '鈴木仁、バッファロー吾郎A、神尾佑、',
               '久保田悠来、磯村勇斗、原沙知絵、池田鉄洋、',
               '高嶋政伸、黒木瞳 etc',
               '原作 ： ソ・ヒョンギョン「TWO WEEKS」',
               '脚本 ： 山浦雅大',
               'Ｐ ： 岡光寛子、白石裕菜、平部隆明',
               '演出 ： 本橋圭太、木内健人 etc',
               '主題歌 ： 三浦春馬『Fight for your heart』',
               'OP曲 ： 04 Limited Sazabys『Montage』'],
 'あなたの番です': ['原田知世、田中圭、',
             '西野七瀬、浅香航大、奈緒、山田真歩、',
             '三倉佳奈、大友花恋、金澤美穂、坪倉由幸、',
             '中尾暢樹、小池亮介、井阪